In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pprint
import json
import re
import selenium
import time
import matplotlib.pyplot as plt

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [ ]:
driver = webdriver.Chrome()

url = "https://understat.com/league/EPL"


driver.get(url)

def get_match_numbers(webdriver):
    mi = webdriver.find_elements_by_class_name("match-info")
    fixture_nums = []
    for i in range(len(mi)):
        href = mi[i].get_attribute("href")
        fixture_nums.append(href[href.find("match/")+6:])
    return fixture_nums


all_match_ids = []
for i in range(10000):
    try:
        all_match_ids = all_match_ids + get_match_numbers(driver)
    except:
        pass
    time.sleep(6)
    button = driver.find_element_by_class_name("calendar-prev")
    button.click()
    print(len(all_match_ids))
    time.sleep(6)
    if len(all_match_ids)>380:
        break
    
    
    

In [ ]:
all_match_ids = list(set(all_match_ids))

In [ ]:
all_match_ids

In [ ]:
fixture_info = dict()

In [ ]:
all_match_ids[0]

In [ ]:
url = "https://understat.com/match/" + "14800"
driver.get(url)

In [ ]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [ ]:
soup.find_all("title")

In [ ]:
fixture_info = dict()

for x in all_match_ids[:4]:
    print(x)
    url = "https://understat.com/match/" + x
    driver.get(url)
    time.sleep(5)
    
    match_info = dict()

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    title = soup.find_all("title")
    print(title)
    data1name = driver.find_element_by_xpath("/html/body/div[1]/div[3]/div[4]/div/div[1]/div/label[1]")

    data1 = soup.find_all("table")

    button = driver.find_element_by_xpath("/html/body/div[1]/div[3]/div[4]/div/div[1]/div/label[2]")
    button.click()
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    data2 = soup.find_all("table")

    data2name = button.text

    match_info["away_team_stats"] = str(data2)
    match_info["home_team_stats"] = str(data1)
    title = str(title)
    match_info["title_details"] = title
    match_info["date"] = title[title.find("(")+1:title.find(")")]
    match_info["home_team"] = title[title.find(">")+1:title.find(" ")]
    match_info["away_team"] = title[title.find("-")+4:title.find("(")-1]

    fixture_info[x] = match_info
    print(len(list(fixture_info.keys())))
    print("-------------")

In [ ]:
fixture_info

In [ ]:
data = json.dumps(fixture_info)

In [ ]:
with open('EPL_20_21.json', 'w') as outfile:
    json.dump(data, outfile)

# Done it! Now for turning html into tables, and joining stuff together

In [ ]:
import matplotlib.pyplot as plt


with open('EPL_20_21___old.json', 'r') as json_file:
    json_data = json.load(json_file)
    json_file.close()

df = json.loads(json_data)

fixture_ids = list(df.keys())

def fixture_data_ETL(fix_id):
    
    date = df[fix_id]["date"]
    
    home = pd.read_html(df[fix_id]["home_team_stats"])[0]
    
    try:
        home["xG"] = home["xG"].apply(lambda x: float(x[0:4]))
    except:
        home["xG"] = home["xG"].apply(lambda x: float(x))
    
    try:
        home["xA"] = home["xA"].apply(lambda x: float(x[0:4]))
    except:
        home["xA"] = home["xA"].apply(lambda x: float(x))
    
    home["team"] = df[fix_id]["home_team"]
    home["opposition"] = df[fix_id]["away_team"]
    home["home_or_away"] = "home"
    
    home["team_Sh"] = home.iloc[-1,:]["Sh"]
    home["team_G"] = home.iloc[-1,:]["G"]
    home["team_KP"] = home.iloc[-1,:]["KP"]
    home["team_A"] = home.iloc[-1,:]["A"]
    home["team_xG"] = home.iloc[-1,:]["xG"]
    home["team_xA"] = home.iloc[-1,:]["xA"]



    away = pd.read_html(df[fix_id]["away_team_stats"])[0]
    
    try:
        away["xG"] = away["xG"].apply(lambda x: float(x[0:4]))
    except:
        away["xG"] = away["xG"].apply(lambda x: float(x))
    
    try:
        away["xA"] = away["xA"].apply(lambda x: float(x[0:4]))
    except:
        away["xA"] = away["xA"].apply(lambda x: float(x))
    
    away["team"] = df[fix_id]["away_team"]
    away["opposition"] = df[fix_id]["home_team"]
    away["home_or_away"] = "away"
    
    away["team_Sh"] = away.iloc[-1,:]["Sh"]
    away["team_G"] = away.iloc[-1,:]["G"]
    away["team_KP"] = away.iloc[-1,:]["KP"]
    away["team_A"] = away.iloc[-1,:]["A"]
    away["team_xG"] = away.iloc[-1,:]["xG"]
    away["team_xA"] = away.iloc[-1,:]["xA"]
    
    away = away.iloc[:-1,1:]
    home = home.iloc[:-1,1:]
    
    home["date"] = date
    away["date"] = date


    away_team_stats = away[["team_Sh", "team_G", "team_KP", "team_A", "team_xG", "team_xA"]].rename(columns={
        "team_Sh":"opponent_Sh",
        "team_G":"opponent_G",
        "team_KP":"opponent_KP",
        "team_A":"opponent_A",
        "team_xG":"opponent_xG",
        "team_xA":"opponent_xA",
    }).iloc[0:11,:]
    
    home_team_stats = home[["team_Sh", "team_G", "team_KP", "team_A", "team_xG", "team_xA"]].rename(columns={
        "team_Sh":"opponent_Sh",
        "team_G":"opponent_G",
        "team_KP":"opponent_KP",
        "team_A":"opponent_A",
        "team_xG":"opponent_xG",
        "team_xA":"opponent_xA",
    }).iloc[0:11,:]
    
    home = pd.concat([home, away_team_stats], axis=1).ffill(axis=0)
    away = pd.concat([away, home_team_stats], axis=1).ffill(axis=0)
    
    fixture_df = pd.concat([home,away]).reset_index().iloc[:,1:]
    
    fixture_df["title_details"] = df[fix_id]["title_details"]
    
    return fixture_df


big_df = fixture_data_ETL(fixture_ids[0])

for n in range(len(fixture_ids[1:])):
    fix_id = fixture_ids[1:][n]
    if n%25 == 0:
        print(n)
    big_df = pd.concat([big_df, fixture_data_ETL(fix_id)]).reset_index().iloc[:,1:]

big_df["month"] = big_df["date"].apply(lambda x: x[:x.find(" ")])

big_df["day"] = big_df["date"].apply(lambda x: x[x.find(" "):][1:3])

big_df["year"] = big_df["date"].apply(lambda x: x[x.find(" "):][4:])

month_mapper = {"January":1,"February":2,"March":3,"April":4,"May":5,"June":6,"July":7,"August":8,
               "September":9,"October":10,"November":11,"December":12}

big_df["month"] = big_df["month"].map(month_mapper)

big_df["datetime"] = pd.to_datetime(big_df[["year", "month","day"]])


In [ ]:
def team_name_generator(title_instance, homeaway_instance):
    titled = title_instance
    dig_ind = re.search(r"\d", titled).start()
    name1 = titled[8:dig_ind-1]
    first = re.search(r"\-", titled).start()
    second= re.search(r"\(", titled).start()
    name2 = titled[first+4:second-1]
    
    if homeaway_instance == "home":
        team = name1
        opponent = name2
    else:
        team = name2
        opponent = name1
    return (team, opponent)

In [ ]:
teams = []
opps = []
for i in range(len(big_df)):
    thingy = team_name_generator(big_df["title_details"][i], big_df["home_or_away"][i])
    teams.append(thingy[0])
    opps.append(thingy[1])

In [ ]:
big_df["team_fixed"] = teams
big_df["opposition_fixed"] = opps

In [ ]:
big_df.to_csv("epl2021_new.csv")

In [ ]:
big_df = pd.read_csv("epl2021_new.csv")

In [ ]:
testo = big_df[big_df["Player"] == "Vladimir Coufal"]

fig,ax = plt.subplots(figsize=(20,6))
testo = testo.sort_values(by="datetime").reset_index().iloc[:,1:]

#testo["team_net_xG"] = testo["team_xG"] - testo["opponent_xG"]
testo["rolling"] = testo["team_xG"].rolling(window=5).mean()
#testo["change_in_rolling"] = testo["rolling"] - testo["rolling"].shift(1)

#plt.plot(testo.index, testo["team_xG"])
#plt.plot(testo.index, testo["team_xG"]-testo["opponent_xG"])

#plt.scatter(testo.index, testo["team_xG"])
plt.plot(testo.index, testo["team_xG"])
plt.plot(testo.index, testo["opponent_xG"])
plt.bar(testo.index, testo["team_G"], alpha=0.3, width=0.3)
plt.bar(testo.index, testo["opponent_G"], alpha=0.7, width=0.1)
plt.plot(testo.index, testo["team_xG"]>testo["opponent_xG"])
plt.legend(["a","b","c","d"]);


In [ ]:
# Plan

# Generate team data: team A, at week 4, have previous created x, conceeded y etc